In [1]:
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from time import time
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
import warnings
import math
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF
import pickle
from sklearn.externals import joblib
warnings.filterwarnings('ignore')
%matplotlib inline

### Load .csv files

In [2]:
# load default data frames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
artists_df = pd.read_csv("artists.csv")
global_median_df = pd.read_csv("global_median.csv")
profiles = pd.read_csv("profiles.csv")

### Create profiles df, imput ages

In [3]:
# CREATES PROFILES DF

# # condition = (profiles_df['age'].isnull()) | (profiles_df['age'] < 95) | (profiles_df['age'] > 0)
# # profiles_df[condition] = np.nan
# profiles_df.loc[profiles_df["age"] < 5 ,'age'] = None
# profiles_df.loc[profiles_df["age"] > 85 ,'age'] = None
# profiles_df.loc[profiles_df["age"].isnull(),'age'] = None

# # calculate mean age per user
# mean_age = profiles_df[~profiles_df['age'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per men
# males_only_df = profiles_df[profiles_df["sex"]=='m']
# mean_male_age = males_only_df[~males_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # calculate mean age per women
# females_only_df = profiles_df[profiles_df["sex"]=='f']
# mean_female_age = females_only_df[~females_only_df['sex'].isnull()]["age"].mean()

# ###################################################################### 

# # get list of unique countries
# countries = profiles_df["country"].unique()

# # calculate mean age per country
# mean_country_age = {}
# for country in countries:
#     tmp_df = profiles_df[profiles_df["country"]==str(country)].copy()
#     mean_country_age[str(country)] = tmp_df["age"].mean()

# ###################################################################### 
    
# # calculate mean age per male per country
# mean_country_male_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='m'].copy()
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)].copy()
#     mean_country_male_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # calculate mean age per female per country
# mean_country_female_age = {}
# for country in countries:
#     tmp_df0 = profiles_df[profiles_df["sex"]=='f'].copy()
#     tmp_df1 = tmp_df0[profiles_df["country"]==str(country)].copy()
#     mean_country_female_age[str(country)] = tmp_df1["age"].mean()

# ###################################################################### 

# # initialize training df
# profiles = profiles_df

# ###################################################################### 

# # create sex indicators
# profiles["male"] = 0
# profiles["female"] = 0
# profiles["sex_missing"] = 0
# profiles.loc[profiles["sex"] =='m', "male"] = 1
# profiles.loc[profiles["sex"] =='f', "female"] = 1
# profiles.loc[profiles["sex"].isnull(), "sex_missing"] = 1

# ######################################################################  

# # create age indicators
# profiles["age_missing"] = 0
# profiles.loc[profiles["age"].isnull(),"age_missing"] = 1

# # fill in mean age: has no gender or country
# profiles.loc[profiles["age_missing"] == 1, "age"] = mean_age

# # fill in mean age: has gender, no country
# #males
# condition = (profiles["sex"] =='m') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_male_age
# # females
# condition = (profiles["sex"] =='f') & (profiles["country"].isnull()) & (profiles["age_missing"] == 1)
# profiles.loc[condition, "age"] = mean_female_age

# # fill in mean age: has country, no gender
# #loop over countries
# for country in countries:
#     condition = (profiles["country"] == country) & (profiles["age_missing"] == 1) & (profiles["sex_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_age[country]

# # fill in mean age: has gender and country
# # loop over countries
# for country in countries:
#     # males
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'm') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_male_age[country]
#     # females
#     condition = (profiles["country"] == country) & (profiles["sex"] == 'f') & (profiles["age_missing"] == 1)
#     profiles.loc[condition, "age"] = mean_country_female_age[country]

# ###################################################################### 
    
# # create country indicators
# profiles["country_missing"] = 0
# profiles.loc[profiles["country"].isnull(), "country_missing"] = 1
# # loop over countries
# for country in countries:
#     # assign 1 if in that country
#     profiles.loc[profiles["country"] == country, country] = 1
#     # assign 0 otherwise
#     profiles.loc[profiles["country"] != country, country] = 0

# ###################################################################### 

# profiles.to_csv("profiles.csv")

# ###################################################################### 

# clean profiles df
profiles.drop("sex",inplace=True,axis=1)
profiles.drop("country",inplace=True,axis=1)
profiles.drop("Unnamed: 0",axis=1,inplace=True)
profiles.set_index("user",inplace=True)
profiles.drop("sex_missing",inplace=True,axis=1)
#profiles.head(1)

In [4]:
profiles.reset_index(inplace=True)

In [5]:
# # if SMALL DF
# profile_features = ["age","male","female","age_missing","United States","Germany","United Kingdom","France","Poland","Brazil","Spain","Italy"]
# num_profile_features = len(profile_features)
# sm_profiles = profiles.loc[:,profile_features]
# sm_profiles.reset_index(inplace=True)
# #sm_profiles.head(1)

### Grab genres for each artist

In [6]:
# load genre data frames and do some cleaning
artists_with_1genre_df = pd.read_csv("artists_with_genres.csv")
artists_with_5genres_df = pd.read_csv("artists_with_top_5_genres.csv")
artists_with_5genres_df.rename(columns = {"num_votes5":"extra"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre5":"num_votes5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes4":"genre5"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre4":"num_votes4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes3":"genre4"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre3":"num_votes3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes2":"genre3"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre2":"num_votes2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"num_votes1":"genre2"}, inplace = True)
artists_with_5genres_df.rename(columns = {"genre1":"num_votes1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"name":"genre1"}, inplace = True)
artists_with_5genres_df.rename(columns = {"artist":"name"}, inplace = True)

# Calculate percent of votes
agn = artists_with_5genres_df[["num_votes1","num_votes2","num_votes3"]]
agn.rename(columns = {"num_votes1":"per_votes1"}, inplace = True)
agn.rename(columns = {"num_votes2":"per_votes2"}, inplace = True)
agn.rename(columns = {"num_votes3":"per_votes3"}, inplace = True)
agn = agn.apply(lambda c: c / c.sum() * 100, axis=1)

del artists_df

# combine dfs together
artists_with_5genres_df = pd.concat([artists_with_5genres_df, agn], axis=1)
#artists_with_5genres_df.head(1)

### Create train df

In [7]:
# Train - Test Split on profiles to ensure users are in one group or another
train, valid = train_test_split(train_df, test_size = .001)
#train, valid = train_test_split(train0, test_size = .1)
#train = train_df.copy()

In [8]:
#add primary genre to train
train["genre1"] = np.nan
train.set_index("artist", inplace=True)
train.update(artists_with_5genres_df)
train.reset_index(inplace=True)

# test["genre1"] = np.nan
# test.set_index("artist", inplace=True)
# test.update(artists_with_5genres_df)
# test.reset_index(inplace=True)

valid["genre1"] = np.nan
valid.set_index("artist", inplace=True)
valid.update(artists_with_5genres_df)
valid.reset_index(inplace=True)

# train0["genre1"] = np.nan
# train0.set_index("artist", inplace=True)
# train0.update(artists_with_5genres_df)
# train0.reset_index(inplace=True)

### compute averages

In [9]:
# calculate global median
global_median = global_median_df.iloc[0,1]
del global_median_df
print global_median

118.0


In [10]:
# calculate user medians
user_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=1)
user_medians = pd.DataFrame(user_medians)
user_medians.rename(columns={0:"user median"},inplace=True)
#user_medians.head(1)

In [11]:
# calculate user means
user_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=1)
user_means = pd.DataFrame(user_means)
user_means.rename(columns={0:"user mean"},inplace=True)
#user_means.head(1)

In [12]:
# calculate artist medians
artist_medians = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').median(axis=0)
artist_medians = pd.DataFrame(artist_medians)
artist_medians.rename(columns={0:"artist median"},inplace=True)
#artist_medians.head(1)

In [13]:
# calculate artist medians
artist_means = train.pivot(index='user',columns='artist',values='plays').reset_index().set_index('user').mean(axis=0)
artist_means = pd.DataFrame(artist_means)
artist_means.rename(columns={0:"artist mean"},inplace=True)
#artist_means.head(1)

In [14]:
# calculate genre medians
genre_medians = train.groupby("genre1").plays.median()
genre_medians = pd.DataFrame(genre_medians)
genre_medians.rename(columns={'plays':"genre median"},inplace=True)
#genre_medians.head(1)

In [15]:
# calculate genre mean
genre_means = train.groupby("genre1").plays.mean()
genre_means = pd.DataFrame(genre_means)
genre_means.rename(columns={'plays':"genre mean"},inplace=True)
#genre_means.head(1)

In [16]:
# calculate genre_user medians
user_genre_medians = train.groupby(["user","genre1"]).median().reset_index()
user_genre_medians.rename(columns={"plays": "user genre median"},inplace=True)
#user_genre_medians.head(1)

In [17]:
# calculate genre_user means
user_genre_means = train.groupby(["user","genre1"]).mean().reset_index()
user_genre_means.rename(columns={"plays": "user genre mean"},inplace=True)
#user_genre_means.head(1)

# Combine Profile and Artist dfs

In [18]:
#pivot training data to have one row per user
train = train.pivot(index='user',columns='artist',values='plays').reset_index()
train = train.fillna(0)

In [19]:
# BIG PROFILE
#get index for later
users = profiles.user
profiles.set_index('user', inplace=True)
# fill in any missing values just in case
profiles[profiles.age.isnull()]=profiles.age.mean()

# standardize profile data
# scaler = StandardScaler()
# profiles_scaled = pd.DataFrame(scaler.fit_transform(profiles))
# cols = profiles.columns
# profiles_scaled.columns = cols
# profiles_scaled.index = users

# combine profile and training data
bigdf = pd.concat([profiles,train],axis=1,join_axes=[profiles.index])
#bigdf.head(1)

num_profile_features = len(profiles.columns)

In [ ]:
# # SMALL PROFILE IF KNN
# #join profiles and train together
# train.set_index('user',inplace = True)
# #train.head(1)

# # get index for later
# users = sm_profiles.user
# sm_profiles.set_index('user', inplace=True)
# # fill in any missing values just in case
# sm_profiles[sm_profiles.age.isnull()]=sm_profiles.age.mean()

# # standardize profile data
# scaler = StandardScaler()
# sm_profiles_scaled = pd.DataFrame(scaler.fit_transform(sm_profiles))
# cols = sm_profiles.columns
# sm_profiles_scaled.columns = cols
# sm_profiles_scaled.index = users

# # combine profile and training data
# bigdf = pd.concat([sm_profiles_scaled,train],axis=1,join_axes=[sm_profiles_scaled.index])
# #bigdf.head(1)

# KNN

In [ ]:
bigdf.fillna(0,inplace=True)
neigh = KNeighborsRegressor(n_neighbors=1000, weights="uniform")
%time neigh.fit(bigdf.iloc[:,:num_profile_features],bigdf.iloc[:,num_profile_features:])

In [ ]:
# use this function in APPLY for a df
def knn_median_predict(test_row,avg):

    # get artist and user info
    artist = test_row[0]
    user = test_row[1]  

    # grab row from bigdf to test
    to_test = bigdf.iloc[bigdf.index == user,:num_profile_features]

    # grab indices of neighbors in bigdf
    idx = neigh.kneighbors(to_test)[1][0]
    predictions = np.array([bigdf.ix[iix,artist] for iix in idx])
    
    # get median from the k=1,3,5 nearest (non-zero) neighbors
    if avg == "median":
        return np.median(predictions[np.nonzero(predictions)][:1]),np.median(predictions[np.nonzero(predictions)][:3]),np.median(predictions[np.nonzero(predictions)][:5])
    else:
         return np.mean(predictions[np.nonzero(predictions)][:1]),np.mean(predictions[np.nonzero(predictions)][:3]),np.mean(predictions[np.nonzero(predictions)][:5])
        

In [ ]:
valid["knn median"] = None
%time valid["knn median"] = valid.iloc[:,:].apply(lambda x: knn_median_predict(x,"median"),axis=1)

In [ ]:
%time valid[['knn1 median', 'knn3 median', 'knn5 median']] = valid['knn median'].apply(pd.Series)

# KMEANS

In [33]:
user_medians.values.reshape([user_medians.shape[0],1]).shape

(233286, 1)

In [43]:
user_medians_r = user_medians.values.reshape([user_medians.shape[0],1])
for i in range(num_profile_features,2245):
    mask = bigdf.iloc[:,i].values == 1102.5
    bigdf.iloc[:,i][mask] = user_medians_r[mask]
    print "\r" + str(i),

2244


In [46]:
scaler = StandardScaler()
bigdf_sc = pd.DataFrame(scaler.fit_transform(bigdf))

In [ ]:
# carry out kmeans
kmeans = KMeans(init = 'k-means++', n_clusters=1000, n_jobs = -1, n_init = 6)
%time kmeans.fit(bigdf_sc)

CPU times: user 17 µs, sys: 66 µs, total: 83 µs
Wall time: 93 µs


In [ ]:
joblib.dump(kmeans, 'kmeans50.pkl') 

In [ ]:
cols = bigdf.columns
centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
centers.columns = cols
centers.head()

### Create test array to compare to kmeans

In [ ]:
closest_centers = kmeans.predict(bigdf_sc)
valid['closest center'] = closest_centers.tolist()
valid['closest center'] = None
valid.set_index("user",inplace=True)
valid.update(train)
valid.reset_index(inplace=True)

In [ ]:
def get_prediction(x):
    artist = x[1]
    return centers.ix[x[5],artist]

In [ ]:
valid["kmeans pred"] = 0
valid["kmeans pred"] = valid.apply(lambda x: get_prediction(x),axis=1)
valid.head()

### Carry out ensemble

In [ ]:
# initialize columns
valid['user genre median'] = np.nan
valid['user median'] = np.nan
valid['artist median'] = np.nan
valid['genre median'] = np.nan
valid['global median'] = np.nan
valid['user mean'] = np.nan
valid['genre mean'] = np.nan
valid['artist mean'] = np.nan
valid['user genre mean'] = np.nan

# add global median
valid['global median'] = global_median

# add user median
valid.set_index('user',inplace=True)
valid.update(user_medians)
valid.reset_index(inplace=True)

# add artist median
valid.set_index('artist',inplace=True)
valid.update(artist_medians)
valid.reset_index(inplace=True)
valid["genre median"].fillna(valid["user median"],inplace=True)

# add genre median
valid.set_index('genre1',inplace=True)
valid.update(genre_medians)
valid.reset_index(inplace=True)
valid["genre median"].fillna(valid["user median"],inplace=True)

# add user genre median
valid.set_index(['user','genre1'],inplace=True)
valid.update(user_genre_medians.set_index(["user","genre1"]))
valid.reset_index(inplace=True)
valid["user genre median"].fillna(valid["user median"],inplace=True)

# add user mean
valid.set_index('user',inplace=True)
valid.update(user_means)
valid.reset_index(inplace=True)

# add artist mean
valid.set_index('artist',inplace=True)
valid.update(artist_means)
valid.reset_index(inplace=True)
valid["genre mean"].fillna(valid["user mean"],inplace=True)

# add genre mean
valid.set_index('genre1',inplace=True)
valid.update(genre_means)
valid.reset_index(inplace=True)
valid["genre mean"].fillna(valid["user mean"],inplace=True)

# add user genre mean
valid.set_index(['user','genre1'],inplace=True)
valid.update(user_genre_means.set_index(["user","genre1"]))
valid.reset_index(inplace=True)
valid["user genre mean"].fillna(valid["user mean"],inplace=True)
valid.head(1)

In [ ]:
print valid.isnull().sum()

In [ ]:
valid.loc[valid['knn1 median'].isnull(),'knn1 median'] = valid.loc[valid['knn1 median'].isnull(),"user median"]
valid.loc[valid['knn3 median'].isnull(),'knn3 median'] = valid.loc[valid['knn3 median'].isnull(),"user median"]
valid.loc[valid['knn5 median'].isnull(),'knn5 median'] = valid.loc[valid['knn5 median'].isnull(),"user median"]

In [ ]:
# weighted ensemble
stacking= linear_model.LinearRegression()

X = np.vstack([valid['knn1 median'].values,valid['knn3 median'].values])
X = np.vstack([X,valid['knn5 median'].values])
X = np.vstack([X,valid['user median'].values])
X = np.vstack([X,valid['user genre median'].values])
X = np.vstack([X,valid['genre median'].values])
X = np.vstack([X,valid['artist median'].values])
X = np.vstack([X,valid['user mean'].values])
X = np.vstack([X,valid['user genre mean'].values])
X = np.vstack([X,valid['artist mean'].values])
X = np.vstack([X,valid['genre mean'].values])


X = X.T

y = valid['plays'].values

stacking.fit(X,y)
stacking.coef_

In [ ]:
valid['ensemble'] =                            \
stacking.coef_[0]*valid['knn1 median']           \
+ stacking.coef_[1]*valid['knn3 median']         \
+ stacking.coef_[2]*valid['knn5 median']         \
+ stacking.coef_[3]*valid['user median']       \
+ stacking.coef_[3]*valid['user genre median'] \
+ stacking.coef_[4]*valid['genre median']      \
+ stacking.coef_[5]*valid['artist median']     \
+ stacking.coef_[6]*valid['user mean']     \
+ stacking.coef_[7]*valid['user genre mean']     \
+ stacking.coef_[8]*valid['artist mean']     \
+ stacking.coef_[9]*valid['genre mean']     \

In [ ]:
valid['AE'] = np.abs(valid['plays']-valid['ensemble']) 
print "MAE is:", valid.AE.mean()

In [ ]:
test = test_df.copy()

In [ ]:
test["genre1"] = np.nan
test.set_index("artist", inplace=True)
test.update(artists_with_5genres_df)
test.reset_index(inplace=True)

In [ ]:
# initialize columns
test['user genre median'] = np.nan
test['user median'] = np.nan
test['artist median'] = np.nan
test['genre median'] = np.nan
test['global median'] = np.nan
test['user mean'] = np.nan
test['genre mean'] = np.nan
test['artist mean'] = np.nan
test['user genre mean'] = np.nan

# add global median
test['global median'] = global_median

# add user median
test.set_index('user',inplace=True)
test.update(user_medians)
test.reset_index(inplace=True)

# add artist median
test.set_index('artist',inplace=True)
test.update(artist_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add genre median
test.set_index('genre1',inplace=True)
test.update(genre_medians)
test.reset_index(inplace=True)
test["genre median"].fillna(test["user median"],inplace=True)

# add user genre median
test.set_index(['user','genre1'],inplace=True)
test.update(user_genre_medians.set_index(["user","genre1"]))
test.reset_index(inplace=True)
test["user genre median"].fillna(test["user median"],inplace=True)

In [ ]:
test["knn1 pred"] = None
%time test["knn1 pred"] = test.apply(lambda x: knn_profile_predict(x,1),axis=1)
test["knn3 pred"] = None
%time test["knn3 pred"] = test.apply(lambda x: knn_profile_predict(x,3),axis=1)
test["knn5 pred"] = None
%time test["knn5 pred"] = test.apply(lambda x: knn_profile_predict(x,5),axis=1)

In [ ]:
test['ensemble'] =                            \
stacking.coef_[0]*test['knn1 pred']         \
+ stacking.coef_[1]*test['knn3 pred']         \
+ stacking.coef_[2]*test['knn5 pred']         \
+ stacking.coef_[3]*test['user median']       \
+ stacking.coef_[4]*test['user genre median'] \
+ stacking.coef_[5]*test['genre median']      \
+ stacking.coef_[6]*test['artist median']     \
# + stacking.coef_[7]*vvalid['global median']     \
# + stacking.coef_[8]*vvalid['user mean']
# + stacking.coef_[9]*vvalid['user genre mean']   \
# + stacking.coef_[10]*vvalid['genre mean']        \
# + stacking.coef_[11]*vvalid['artist mean']      

In [ ]:
output_for_kaggle = test
output_for_kaggle.drop(["plays","closest center", "AE", "genre1","user","artist","knn pred","knn1 pred","knn3 pred","knn5 pred","user genre median","user median", "genre median", "global median","artist median","artist mean","user mean", "genre mean", "user genre mean"],axis=1,inplace=True)
output_for_kaggle.rename(columns={"ensemble":"plays"},inplace = True)
output_for_kaggle.head()